## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Descubrimiento de grupos en los datos

---

En esta libreta se generan los modelos de clusterización sobre el dataset final del escenario para su posterior evaluación y análisis. Se utilizará como herramienta de soporte a mlflow para el registro completo de la experimentación. 

---

In [1]:
# Importación de librerías

import pandas as pd
# Se importa mlflow para registro de la experimentación 
import mlflow
# Se importan los métodos a utilizar para clusterizar
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering, MeanShift
# Se importan las métricas a utilizar para evaluar el proceso
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

from sklearn.cluster import MeanShift, estimate_bandwidth

from sklearn.cluster import SpectralClustering

import warnings

# Ignorar todas las advertencias temporalmente
with warnings.catch_warnings():
    warnings.simplefilter("ignore")


----

##### Lectura del dataset

In [2]:
df = pd.read_csv("../../../data/final/datos_clusterizacion.csv", sep=";")
df.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,...,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,713061558.0,22,35000,3,123.0,59000,16.02,1,36.0,1088.0,...,True,False,False,True,False,False,False,False,True,False
1,768805383.0,21,1000,2,5.0,9600,11.14,0,39.0,1144.0,...,True,False,False,True,False,False,False,True,False,False
2,713982108.0,23,35000,2,4.0,65500,15.23,1,36.0,1887.0,...,True,False,False,True,False,False,False,False,True,False
3,710821833.0,24,35000,4,8.0,54400,14.27,1,54.0,1314.0,...,True,False,False,True,True,False,False,False,False,False
4,769911858.0,21,2500,2,2.0,9900,7.14,1,34.0,1171.0,...,True,False,True,False,False,False,False,True,False,False


----

##### Configuración de la experimentación

In [5]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering2'
exp_id = mlflow.create_experiment(name=exp_name)


### Prueba Nro. 1

En la medida de lo posible la cantidad de clusters va a ser de 2

In [6]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P1: Kmeans - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K2")

    # Se finaliza el registro
    mlflow.end_run()

# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k2 = df.copy()
# Se agregan las labels generadas
df_kmeans_k2['cluster'] = cluster_labels
df_kmeans_k2['cluster'].value_counts()

cluster
0    6040
1    2831
Name: count, dtype: int64

Para contabilizar los elementos por cluster se utilizan las labels generadas

Se pasa a probar otro método (clustering jerárquico)

In [7]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P1: AGClustering - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K2")

    # Se finaliza el registro
    mlflow.end_run()

# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

2024/07/17 00:08:06 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


cluster
1    5953
0    2918
Name: count, dtype: int64

Para contabilizar los elementos por cluster se utilizan las labels generadas

Finalmente, se prueba un método que no tiene parámetros

In [8]:
# Estimar el ancho de banda adecuado
bandwidth = estimate_bandwidth(df, quantile=0.6, n_samples=len(df))

# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P1: MeanShift 0.6"):
    # Selección y configuración de la técnica
    modelo_clusters = MeanShift(bandwidth=bandwidth)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "MS")

    # Se finaliza el registro
    mlflow.end_run()

# Se preserva el conjunto de datos original para evitar problemas
df_ms = df.copy()
# Se agregan las labels generadas
df_ms['cluster'] = cluster_labels
df_ms['cluster'].value_counts()

c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


cluster
0    6017
1    2854
Name: count, dtype: int64

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [9]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P1: Spectral_2_NN"):
    # Selección y configuración de la técnica
    modelo_clusters = SpectralClustering(n_clusters=2, affinity='nearest_neighbors')
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Spectral_2NN")

    # Se finaliza el registro
    mlflow.end_run()


# Se preserva el conjunto de datos original para evitar problemas
df_Spectral_2NN = df.copy()
# Se agregan las labels generadas
df_Spectral_2NN['cluster'] = cluster_labels
df_Spectral_2NN['cluster'].value_counts()

c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
2024/07/17 00:11:48 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Us

cluster
0    8853
1      18
Name: count, dtype: int64

----

### Prueba Nro. 2

- Kmeans


In [10]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P2: Kmeans - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K3")

    # Se finaliza el registro
    mlflow.end_run()

# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k3 = df.copy()
# Se agregan las labels generadas
df_kmeans_k3['cluster'] = cluster_labels
df_kmeans_k3['cluster'].value_counts()

c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


cluster
1    6005
2    1919
0     947
Name: count, dtype: int64

- Clustering Aglomerativo

In [12]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P3: AGClustering - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K3")

    # Se finaliza el registro
    mlflow.end_run()

# Se preserva el conjunto de datos original para evitar problemas
df_agc_k3 = df.copy()
# Se agregan las labels generadas
df_agc_k3['cluster'] = cluster_labels
df_agc_k3['cluster'].value_counts()

2024/07/17 00:11:58 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


cluster
1    5953
0    2073
2     845
Name: count, dtype: int64

In [13]:
# Estimar el ancho de banda adecuado
bandwidth = estimate_bandwidth(df, quantile=0.3, n_samples=len(df))

# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P2: MeanShift 0.3"):
    # Selección y configuración de la técnica
    modelo_clusters = MeanShift(bandwidth=bandwidth)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "MS_03")

    # Se finaliza el registro
    mlflow.end_run()

# Se preserva el conjunto de datos original para evitar problemas
df_ms_03 = df.copy()
# Se agregan las labels generadas
df_ms_03['cluster'] = cluster_labels
df_ms_03['cluster'].value_counts()

c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


cluster
0    6008
1    1945
2     918
Name: count, dtype: int64

In [14]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P: Spectral_3_NN"):
    # Selección y configuración de la técnica
    modelo_clusters = SpectralClustering(n_clusters=3, affinity='nearest_neighbors')
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Spectral_3NN")

    # Se finaliza el registro
    mlflow.end_run()


# Se preserva el conjunto de datos original para evitar problemas
df_Spectral_3NN = df.copy()
# Se agregan las labels generadas
df_Spectral_3NN['cluster'] = cluster_labels
df_Spectral_3NN['cluster'].value_counts()

c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
2024/07/17 00:14:53 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Us

cluster
0    8829
1      24
2      18
Name: count, dtype: int64

### Prueba Nro. 3

In [20]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P3: Kmeans - K=4"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=4)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 4)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K4")

    # Se finaliza el registro
    mlflow.end_run()

# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k4 = df.copy()
# Se agregan las labels generadas
df_kmeans_k4['cluster'] = cluster_labels
df_kmeans_k4['cluster'].value_counts()

c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


cluster
0    5984
1    1331
2     845
3     711
Name: count, dtype: int64

In [21]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P3: AGClustering - K=4"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=4)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 4)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K4")

    # Se finaliza el registro
    mlflow.end_run()

# Se preserva el conjunto de datos original para evitar problemas
df_agc_k4 = df.copy()
# Se agregan las labels generadas
df_agc_k4['cluster'] = cluster_labels
df_agc_k4['cluster'].value_counts()

2024/07/17 01:10:17 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


cluster
1    5953
0    1277
2     845
3     796
Name: count, dtype: int64

In [22]:
# Estimar el ancho de banda adecuado
bandwidth = estimate_bandwidth(df, quantile=0.2, n_samples=len(df))

# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P3: MeanShift 0.2"):
    # Selección y configuración de la técnica
    modelo_clusters = MeanShift(bandwidth=bandwidth)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "MS_02")

    # Se finaliza el registro
    mlflow.end_run()

# Se preserva el conjunto de datos original para evitar problemas
df_ms_02 = df.copy()
# Se agregan las labels generadas
df_ms_02['cluster'] = cluster_labels
df_ms_02['cluster'].value_counts()

c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


cluster
0    5956
1    1520
2     616
3     418
4     361
Name: count, dtype: int64

In [23]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="P3: Spectral_4_NN"):
    # Selección y configuración de la técnica
    modelo_clusters = SpectralClustering(n_clusters=4, affinity='nearest_neighbors')
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Spectral_4NN")

    # Se finaliza el registro
    mlflow.end_run()


# Se preserva el conjunto de datos original para evitar problemas
df_Spectral_4NN = df.copy()
# Se agregan las labels generadas
df_Spectral_4NN['cluster'] = cluster_labels
df_Spectral_4NN['cluster'].value_counts()

c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
2024/07/17 01:13:25 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
c:\Users\Juan Bonilla\Documents\Maestria\Metodologia gestion proyectos y big data\Manejo_proyecto\13MDIB_JBONiILLA\.venv\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Us

cluster
0    8207
3     522
1     106
2      36
Name: count, dtype: int64

In [25]:
df_kmeans_k3.to_csv("../../../data/final/resultados_kmeans_k3.csv", sep=";", index=False)
df_kmeans_k4.to_csv("../../../data/final/resultados_kmeans_k4.csv", sep=";", index=False)